In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
from datetime import date, timedelta
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#SET UP SEABORN
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")

In [ ]:
# creates a dictionary that maps `category_id` to `category`
import json
id_to_category = {}

with open('/kaggle/input/trending-youtube-videos-by-category/videoCategories.json', 'r') as f:
    data = json.load(f)
    for category in data['items']:
        id_to_category[category['id']] = category['snippet']['title']

In [ ]:
#load data
raw_data_2Jul = pd.read_csv('/kaggle/input/trending-youtube-videos-by-category/21.02.07_VN_videos.csv')
raw_data_1Jul = pd.read_csv('/kaggle/input/trending-youtube-videos-by-category/21.01.07_VN_videos.csv')

#map category to raw_data
raw_data_2Jul['categoryId'] = raw_data_2Jul.categoryId.astype(str)
raw_data_1Jul['categoryId'] = raw_data_1Jul.categoryId.astype(str)

raw_data_2Jul.insert(5, 'category', raw_data_2Jul.categoryId.map(id_to_category))
raw_data_1Jul.insert(5, 'category', raw_data_1Jul.categoryId.map(id_to_category))

#conver to date format
def convert_todate(df):
    df['publishedAt'] = pd.to_datetime(df['publishedAt']).dt.date
    df['trending_date'] = pd.to_datetime(df['trending_date'],format='%y.%d.%m').dt.date

convert_todate(raw_data_2Jul)
convert_todate(raw_data_1Jul)

# drop duplicate
raw_data_2Jul = raw_data_2Jul.drop_duplicates(subset='video_id', keep="last")
raw_data_1Jul = raw_data_1Jul.drop_duplicates(subset='video_id', keep="last")

# # **1. CREATING df: DATA**

In [ ]:
today = data.trending_date.max()
yesterday = today - datetime.timedelta(days=1)

In [ ]:
data = pd.concat([raw_data_1Jul,raw_data_2Jul])

In [ ]:
data_trending_today = data[data['trending_date'] == today]
data_trending_today.rename(columns={'view_count':'view_count_t',
                                   'likes':'likes_t',
                                   'dislikes':'dislikes_t',
                                   'comment_count' :'comment_count_t'},
                          inplace = True)

In [ ]:
# CREATE DATA_TRENDING
data_trending = pd.merge(data_trending_today,data[data['trending_date'] == yesterday].iloc[:,[0,9,10,11,12]],on=['video_id'],how='left')

In [ ]:
data_trending['growth_view'] = round(((data_trending['view_count_t'] - data_trending['view_count'])/data_trending['view_count'])*100,2)
data_trending['growth_likes'] = round(((data_trending['likes_t'] - data_trending['likes'])/data_trending['likes'])*100,2)
data_trending['growth_dislikes'] = round(((data_trending['dislikes_t'] - data_trending['dislikes'])/data_trending['dislikes'])*100,2)
data_trending['growth_comt'] = round(((data_trending['comment_count_t'] - data_trending['comment_count'])/data_trending['comment_count'])*100,2)
data_trending.growth_comt.replace({float(np.inf):100}, inplace = True)

data_trending['total_growth'] = data_trending['growth_view'] + data_trending['growth_likes'] + 0.5*data_trending['growth_dislikes'] + data_trending['growth_comt']

In [ ]:
data_trending.

In [ ]:
sns.heatmap(df)

In [ ]:
# mean() total_growth per category
data_trending.groupby('category').growth_view.mean().sort_values()

# OLD VERSION

In [ ]:
#LOAD DATA
my_filepath = "../input/youtube-new/USvideos.csv"
raw_data = pd.read_csv(my_filepath)

#DROP DUPLICATES
raw_data = raw_data.drop_duplicates()

#CONVERT DATETIME FORMAT "%Y/%m/%d"
raw_data['publish_time'] = pd.to_datetime(raw_data.publish_time).dt.strftime("%Y/%m/%d").astype('datetime64')
raw_data['trending_date']= pd.to_datetime(raw_data.trending_date, format='%y.%d.%m').dt.strftime("%Y/%m/%d").astype('datetime64')

raw_data.head(5)

In [ ]:
#channel having the highest number of trending videos
channel_trending = raw_data.groupby('channel_title').agg(count_video =('video_id', 'nunique')).sort_values(by=['count_video'],  ascending = False).reset_index()
channel_trending.head(10)

In [ ]:
trending_date.time_to_becometrending

In [ ]:
#TRENDING DATE CALCULATION
#latest_trending_date= raw_data.groupby(['video_id','title','publish_time']).trending_date.max()
trending_date= raw_data.groupby(['video_id','title','publish_time']).agg(latest_trending_date =('trending_date', 'max'),becometrending_date = ('trending_date', 'min') ).reset_index()
trending_date['time_to_becometrending'] = (trending_date['latest_trending_date'] - trending_date['publish_time']).apply(lambda x: x.days)
trending_date['trending_duration'] = (trending_date['latest_trending_date'] - trending_date['becometrending_date']).apply(lambda x: x.days)
trending_date.head(5)

In [ ]:
trending_date.trending_duration.hist()

In [ ]:
trending_date.time_to_becometrending.hist()

In [ ]:
trending_date[trending_date['time_to_becometrending'] > 2000]

In [ ]:
sns.jointplot(x='views',y='likes',data=raw_data,kind='reg')

In [ ]:
r1 = np.corrcoef(raw_data['views'], raw_data['likes'])
r1

In [ ]:
sns.jointplot(x='views',y='dislikes',data=raw_data,kind='reg')

In [ ]:
r2 = np.corrcoef(raw_data['views'], raw_data['dislikes'])
r2